In [27]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [28]:
def load_nifti_files(file_paths):
    data = []
    for file_path in file_paths:
        img = nib.load(file_path)
        data.append(img.get_fdata())
    return np.array(data)

In [31]:
# Downloading required dataset files
import gdown
hdr_ids = ["1V-uHcMuSAe39jETKtb-3oVkDJ7QCIF-y", "15mJ8CxMhyhfbzTVPpF9EMG0rdQClUJ8b", "1rNqIks6jCj-APTAf85eyAzOOQewpsKtH", "1YQ-vQO2tTTcw1bmCkt4COpeWrblsHikb"]
i = 1
for id in hdr_ids:
  url = f'https://drive.google.com/uc?id={id}'
  output = f"/content/{i}.hdr"
  gdown.download(url, output, quiet=False)
  i += 1

img_ids = ["1BBUdxoVWf7bXcoist47Eotlaf6PGrP25", "1ZErMdV5s4EtPxjKYMkZNMWQ7FxUk-6DN", "1gJ3CBEi1uqUy8oBVe7i2m9XP7252vGuV", "1SvwMt0vaK2pzlzR9i80QjkPzut6zFsTE"]
j = 1
for id in img_ids:
  url = f'https://drive.google.com/uc?id={id}'
  output = f"/content/{j}.img"
  gdown.download(url, output, quiet=False)
  j += 1

Downloading...
From: https://drive.google.com/uc?id=1V-uHcMuSAe39jETKtb-3oVkDJ7QCIF-y
To: /content/1.hdr
100%|██████████| 348/348 [00:00<00:00, 219kB/s]
Downloading...
From: https://drive.google.com/uc?id=15mJ8CxMhyhfbzTVPpF9EMG0rdQClUJ8b
To: /content/2.hdr
100%|██████████| 348/348 [00:00<00:00, 908kB/s]
Downloading...
From: https://drive.google.com/uc?id=1rNqIks6jCj-APTAf85eyAzOOQewpsKtH
To: /content/3.hdr
100%|██████████| 348/348 [00:00<00:00, 1.01MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YQ-vQO2tTTcw1bmCkt4COpeWrblsHikb
To: /content/4.hdr
100%|██████████| 348/348 [00:00<00:00, 1.21MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BBUdxoVWf7bXcoist47Eotlaf6PGrP25
To: /content/1.img
100%|██████████| 33.6M/33.6M [00:00<00:00, 95.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZErMdV5s4EtPxjKYMkZNMWQ7FxUk-6DN
To: /content/2.img
100%|██████████| 33.6M/33.6M [00:00<00:00, 98.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gJ3CBEi1uqUy8oBVe

In [32]:
# Load the MRI scans and labels
labels = [0, 1, 0, 0]
mri_file_paths = ["/content/1.hdr", "/content/2.hdr", "/content/3.hdr", "/content/4.hdr"]
mri_scans = load_nifti_files(mri_file_paths)
labels = np.array(labels)

# Normalize the MRI scans to have values between 0 and 1
mri_scans = mri_scans / np.max(mri_scans)

print("Shape of MRI scans:", mri_scans.shape)

Shape of MRI scans: (4, 256, 256, 128, 1)


In [33]:
# Preprocessing the data to reduce training load
def resize_image(image, new_shape):
    old_shape = np.array(image.shape)
    ratio = new_shape / old_shape
    return zoom(image, ratio, mode='nearest')

resized_scans = np.array([resize_image(scan, (128, 128, 64, 1)) for scan in mri_scans])
print("Shape of preprocessed MRI scans:", resized_scans.shape)

Shape of preprocessed MRI scans: (4, 128, 128, 64, 1)


In [34]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(resized_scans, labels, test_size=0.2, random_state=42)

# Reshape the data to include the channel dimension (required for CNN)
x_train = x_train.reshape((-1, 128, 128, 64, 1))
x_test = x_test.reshape((-1, 128, 128, 64, 1))

In [35]:
def create_model():
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(128, 128, 64, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [36]:
model = create_model()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
1/1 [==============================] - 18s 18s/step - loss: 0.7050 - accuracy: 0.0000e+00 - val_loss: 85.4571 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 21s 21s/step - loss: 6.5039e-33 - accuracy: 1.0000 - val_loss: 157.5114 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 17s 17s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 227.7416 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 297.1785 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 365.3057 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 431.4995 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_